In [ ]:
# MAXpool
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, GlobalMaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.applications import ResNet50, ResNet152, InceptionV3, InceptionResNetV2, Xception, DenseNet121, DenseNet169, DenseNet201
from tensorflow.keras.applications.resnet import preprocess_input as resnet_preprocess_input
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess_input
from tensorflow.keras.applications.xception import preprocess_input as xception_preprocess_input
from tensorflow.keras.applications.densenet import preprocess_input as densenet_preprocess_input
from tensorflow.keras.callbacks import LearningRateScheduler
import math

# Set random seed for reproducibility
SEED = 42
tf.random.set_seed(SEED)

# Check GPU availability
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Directories for training and testing data
train_dir = "/kaggle/input/d/shahriarraj/skin-lesion-input-dataset-123/Skin Lesion Input Dataset/Train_Set_Images"
test_dir = "/kaggle/input/d/shahriarraj/skin-lesion-input-dataset-123/Skin Lesion Input Dataset/Test_Set_Images"

IMG_SIZE = (224, 224)
BATCH_SIZE = 64

# Data generators
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=SEED
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Function to build models dynamically
def build_model(model_name):
    if model_name == 'ResNet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        preprocess_func = resnet_preprocess_input
    elif model_name == 'ResNet152':
        base_model = ResNet152(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        preprocess_func = resnet_preprocess_input
    elif model_name == 'InceptionV3':
        base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        preprocess_func = inception_preprocess_input
    elif model_name == 'InceptionResNetV2':
        base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        preprocess_func = inception_preprocess_input
    elif model_name == 'Xception':
        base_model = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        preprocess_func = xception_preprocess_input
    elif model_name == 'DenseNet121':
        base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        preprocess_func = densenet_preprocess_input
    elif model_name == 'DenseNet169':
        base_model = DenseNet169(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        preprocess_func = densenet_preprocess_input
    elif model_name == 'DenseNet201':
        base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        preprocess_func = densenet_preprocess_input

    
    # Build the model
    model = Sequential([
        base_model,
        GlobalMaxPooling2D(),
        Dense(64, activation='relu'),
        Dropout(0.2),  # Set dropout rate to 0.2
        Dense(train_generator.num_classes, activation='softmax')
    ])
    return model

# List of models to train
models_to_train = ['Xception']

# Training loop
for model_name in models_to_train:
    print(f"\nTraining {model_name}...\n")
    model = build_model(model_name)

    # Compile the model
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    # Learning Rate Scheduler
    def step_decay(epoch):
        initial_lr = 0.001
        drop_rate = 0.1
        epochs_drop = 5
        new_lr = initial_lr * math.pow(drop_rate, math.floor((1 + epoch) / epochs_drop))
        return new_lr
    
    lr_scheduler = LearningRateScheduler(step_decay)
    
    # Train the model
    history = model.fit(
        train_generator,
        epochs=20,
        validation_data=test_generator,
        callbacks=[lr_scheduler],
        verbose=1
    )

    # Evaluate the model
    train_loss, train_acc = model.evaluate(train_generator)
    test_loss, test_acc = model.evaluate(test_generator)

    print(f"{model_name} Training Accuracy: {train_acc*100:.2f}%")
    print(f"{model_name} Testing Accuracy: {test_acc*100:.2f}%")


Num GPUs Available:  2
Found 5823 images belonging to 7 classes.
Found 1459 images belonging to 7 classes.

Training Xception...

Epoch 1/20


In [4]:
import os

# Define the Kaggle input directory
input_dir = "/kaggle/input"

# Walk through the directory and list all files
for root, dirs, files in os.walk(input_dir):
    print(f"Directory: {root}")
    for file in files:
        print(f" - {file}")

Directory: /kaggle/input
Directory: /kaggle/input/d
Directory: /kaggle/input/d/shahriarraj
Directory: /kaggle/input/d/shahriarraj/skin-lesion-input-dataset-123
Directory: /kaggle/input/d/shahriarraj/skin-lesion-input-dataset-123/Skin Lesion Input Dataset
Directory: /kaggle/input/d/shahriarraj/skin-lesion-input-dataset-123/Skin Lesion Input Dataset/Test_Set_Images
Directory: /kaggle/input/d/shahriarraj/skin-lesion-input-dataset-123/Skin Lesion Input Dataset/Test_Set_Images/mel
 - ISIC_0033152.jpg
 - ISIC_0028075.jpg
 - ISIC_0033878.jpg
 - ISIC_0032987.jpg
 - ISIC_0024554.jpg
 - ISIC_0027620.jpg
 - ISIC_0026996.jpg
 - ISIC_0033037.jpg
 - ISIC_0029843.jpg
 - ISIC_0032532.jpg
 - ISIC_0029480.jpg
 - ISIC_0033717.jpg
 - ISIC_0031239.jpg
 - ISIC_0033872.jpg
 - ISIC_0029495.jpg
 - ISIC_0030171.jpg
 - ISIC_0033999.jpg
 - ISIC_0028666.jpg
 - ISIC_0028215.jpg
 - ISIC_0025155.jpg
 - ISIC_0033905.jpg
 - ISIC_0025531.jpg
 - ISIC_0031350.jpg
 - ISIC_0024958.jpg
 - ISIC_0030129.jpg
 - ISIC_0026774.jpg